# DC Bridge -- Solving Linear Systems

## Crash course in linear algebra for equation solving


Once a circuit gets to a certain level of complexity, solving the loop and junction rule equations for all of the voltage drops and currents becomes very cumbersome.  This is where linear algebra can help us, because it is designed for solving systems of connected equations.  In case you haven't seen this before, here's a quick conceptual primer on how we solve a whole system of related equations at once using matrix methods.  

Let's say we have the following circuit and we want to find the current through each of the resistors:

<img src="test_parallel_circuit.png" width=200 height=200 />
    
We'll use the loop rule and we'll need three equations, because the currents through the three branches are our three unknowns.  Let's assume that the current goes left in the top, right in the middle, and right in the bottom.
    
\begin{eqnarray}
\textrm{Top loop, counter-clockwise: }& 12 V - R_2 I_2 + 3 V - R_1 I_1 &= 0\\
\textrm{Bottom loop, counter-clockwise: } & 9 V - 3 V + R_2 I_2 - R_3 I_3 &= 0\\
\textrm{Outer loop, counter-clockwise: }& 12 V - R_3 I_3 + 9V - R_1 I_1 &= 0\\
\end{eqnarray}

Let's rearrange this so that all of the constant voltages are on the right hand side and there is a term in each equation for each current:

\begin{eqnarray}
\textrm{Top loop, counter-clockwise: }&  R_1 I_1 + R_2 I_2 + 0 \cdot I_3&= 15 V\\
\textrm{Bottom loop, counter-clockwise: } & 0\cdot I_1 - R_2 I_2 + R_3 I_3 &= 6 V\\
\textrm{Outer loop, counter-clockwise: }& R_1\cdot I_1 + 0\cdot I_2 + R_3 I_3 &= 21 V\\
\end{eqnarray}

Now, we can write this as a matrix equation:

$$
\begin{bmatrix} R_1 & R_2 & 0 \\ 0 & -R_2 & R_3 \\ R_1 & 0 & R_3 \end{bmatrix}
\begin{bmatrix}I_1 \\ I_2 \\ I_3\end{bmatrix} 
= \begin{bmatrix}15 V \\ 6 V \\ 21 V\end{bmatrix}
\;\;\; \textrm{ or } \;\;\; \mathcal{{RI = V}}
$$

We know the values of the resistances, so what we actually need to do is _divide_ $V/R$ to find the currents.  
Since we can't divide matrices, we need to multiply both sides on the left by the _inverse_ of the resistor matrix:

$$
\mathcal{R^{-1} R I = R^{-1} V}\;\;\; \textrm{ or } \;\;\; \mathcal{I = R^{-1}V}
$$

You can calculate the inverse by hand, but there are good algorithms for it that make computers particularly useful for the task.  Here, we're going to use the `numpy.linalg` library to solve the problem quickly.  The solver assumes that our equation is in the form $\mathcal{AX = B}$, takes $\mathcal{A}$ and $\mathcal{B}$ as input parameters, and solves for $\mathcal{X}$:

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# define our A and B matrices, which in our case are the resistor matrix and the voltage matrix
R1 = 47. #Ohms
R2 = 100. #Ohms
R3 = 47. #Ohms
resistor_matrix = np.array([[R1, R2, 0], [0, -R2, R3], [R1, 0, R3]]) #Ohms
voltage_matrix = np.array([[15.], [6.], [21.]]) #Volts

# now we give these to the solver and the result is the X matrix we are after
current_matrix = np.linalg.solve(resistor_matrix, voltage_matrix)

# we can look at the solution to see the values for the currents
print(current_matrix,"\n")

# here's a little trick for printing out the values nicely
for ii, current in enumerate(current_matrix):
    disp_string = "I{}"+ " = " + "{:4.2f} A"
    print(disp_string.format(ii+1,current[0]))
    

LinAlgError: Singular matrix

Oops!  A singular matrix usually means that we don't have a complete set of information.  In the example above, the first two loop rule equations sum to the third one!  This means that we only have two independent equations, not three.  We need a junction rule.

Let's say we I want to use a junction rule instead of the outer loop for the circuit above:

<img src="test_parallel_circuit.png" width=200 height=200 />
    
    
\begin{eqnarray}
\textrm{Top loop, counter-clockwise: }& 12 V - R_2 I_2 + 3 V - R_1 I_1 &= 0\\
\textrm{Bottom loop, counter-clockwise: } & 9 V - 3 V + R_2 I_2 - R_3 I_3 &= 0\\
\textrm{Left side junction: }& I_1 - I_2 - I_3 &= 0\\
\end{eqnarray}

To make sure all of our equations have the same units (since they will become part of a single matrix equation), let's multiply that last equation through by $R_1$:

\begin{eqnarray}
\textrm{Top loop, counter-clockwise: }&  R_1 I_1 + R_2 I_2 + 0 \cdot I_3&= 15 V\\
\textrm{Bottom loop, counter-clockwise: } & 0\cdot I_1 - R_2 I_2 + R_3 I_3 &= 6 V\\
\textrm{Left side junction: }& R_1 I_1 - R_1 I_2- R_1 I_3 &= 0 V\\
\end{eqnarray}

Now, our matrix equation looks like:

$$
\begin{bmatrix} R_1 & R_2 & 0 \\ 0 & -R_2 & R_3 \\ R_1 & -R_1 & -R_1 \end{bmatrix}
\begin{bmatrix}I_1 \\ I_2 \\ I_3\end{bmatrix} 
= \begin{bmatrix}15 V \\ 6 V \\ 0 V\end{bmatrix}
\;\;\; \textrm{ or } \;\;\; \mathcal{{RI = V}}
$$

We are still solving the same matrix equation, but with a new resistance matrix and a new voltage vector.


In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# define our A and B matrices, which in our case are the resistor matrix and the voltage matrix
R1 = 47. #Ohms
R2 = 100. #Ohms
R3 = 47. #Ohms
resistor_matrix = np.array([[R1, R2, 0], [0, -R2, R3], [R1, -R1, -R1]]) #Ohms
voltage_matrix = np.array([[15.], [6.], [0]]) #Volts

# now we give these to the solver and the result is the X matrix we are after
current_matrix = np.linalg.solve(resistor_matrix, voltage_matrix)

# we can look at the solution to see the values for the currents
print(current_matrix,"\n")

# here's a little trick for printing out the values nicely
for ii, current in enumerate(current_matrix):
    disp_string = "I{}"+ " = " + "{:4.2f} A"
    print(disp_string.format(ii+1,current[0]))
    

[[0.24162288]
 [0.03643725]
 [0.20518563]] 

I1 = 0.24 A
I2 = 0.04 A
I3 = 0.21 A


## The Wheatstone Bridge

Let's say you want to measure a very low resistance, like that of a small piece of wire.  Your digital voltmeter likely won't have the necessary resolution to do so (our DMMs have a minimum count of 0.1$\Omega$).  

There is an old technique based on balancing two voltage dividers that can allow very sensitive resistance measurement.  The setup is called a Wheatstone bridge, and is shown in Fig. 2.65 from the Sherz and Monk:

<img src="PEFI_fig_2p65.png" width=400/>

The voltage across $R_4$ reflects any difference in the *ratios* of the two dividers.  When $R_2/R_5$ is equal to $R_3/R_6$, there is no voltage difference across $R_4$ and no current flows there.  To measure an unknown resistance, you can replace one leg of the divider with the object to be measured (e.g. $R_3$) and a precisely variable resistor (e.g. $R_6$), like a winding of wire with a sliding contact.  It is much easier to measure zero current than to determine a precise value for the current (which requires more calibration), so the technique is to "balance" the bridge by adjusting $R_6$ until the ratios are equal.  Then, the value of $R_3$ can be calculated precisely from the divider ratio (easily measured) and the value of $R_6$.  Instead of relying on this simplicity, however, we're going to use this circuit to illustrate the use of matrix algebra to solve the whole system at once (as in the text). 

We don't have precision, adjustable, wirewound resistors lying around any more.  If we replace $R_6$ with a fixed resistance, we can calibrate the bridge so that $R_3$ can be determined by looking at the voltage across $R_4$. This means we need to solve the whole system with known resistances, as in the text.

#### Exercise 1

Use the known set of resistances below to calibrate the bridge for a 10V battery.  Solve the bridge system for all of the currents and then calculate the voltage across the center resistor ($R_4$).  What does this tell you about the calibration factor of the bridge ($V/\Omega$)?

\begin{eqnarray}
R_1 & = 1\Omega\\
R_2 & = 100\Omega\\
R_3 & = 0.05\Omega\\
R_4 & = 100\Omega\\
R_5 & = 100 k\Omega\\
R_6 & = 10\Omega\\
\end{eqnarray}